In [119]:
import numpy as np
import networkx as nx
import json
from collections import Counter
import random
import sim
import operator
from heapq import nlargest

In [120]:
# Global Variables
n = 10

In [121]:
def get_graphs(filenames):
    filepath = '/Users/velis.christ/Downloads/'
    graphs = []
    adj_lists = []
    for filename in filenames:
        with open(filepath+filename+".json") as f:
            adj_list = json.load(f)
        # Create a graph
        G = nx.Graph(adj_list)
        graphs.append(G)
        adj_lists.append(adj_list)
    return graphs, adj_lists

In [122]:
def retrieve_strat_filenames(graph_filename, team_names):
    res = []
    for name in team_names:
        strat_filename = graph_filename + "-" + name + ".json"
        res.append(strat_filename)
    return res

def get_strategies(filenames):
    filepath = '/Users/velis.christ/Downloads/'
    strategies = []
    for filename in filenames:
        with open(filepath+filename) as f:
            strategy = json.load(f)
        strategies.append(strategy)
    return strategies

In [123]:
def compare_all(adj_list, best_nodes, all_test_nodes):
    results = []
    # print(all_test_nodes)
    for test_nodes in all_test_nodes:
        node_mappings = dict()
        node_mappings["Best"] = best_nodes
        node_mappings["Test"] = test_nodes
        # print(node_mappings)
        result = sim.run(adj_list, node_mappings)
        results.append(result)
    return results

In [124]:
def best_strategy(graphs, adj_lists, strategies):
    ''' Find the best strategy (mix of centrality measures)
        given a set of graphs and competing strategies.
        The graphs are of the same type (e.g., SNAP)
        
        Inputes:
        graphs: list of nx.Graph()
        adj_lists: list of dict() adj_list
        strategies: list of lists of lists '''

    # Compute centralities for each graph
    eig_cent = []
    close_cent = []
    triangles = []
    for i in range(len(graphs)):
        G = graphs[i]
        eig_cent.append(nx.eigenvector_centrality(G))
        close_cent.append(nx.closeness_centrality(G))
        triangles.append(nx.triangles(G))

    # Find best coefficients by trying many different coefficients
    coeffs = []
    scores = []
    num_beats = []
    for alpha in np.linspace(0,1,num=10):
        for beta in np.linspace(0,1,num=10):
            coeffs.append((alpha, beta))
            our_node_mapping = {}
            score = 0
            num_beat = 0
            # Iterate over each graph to compute the score of the current strategy
            for i in range(len(graphs)):
                G = graphs[i]
                curr_eig_cent = eig_cent[i]
                curr_close_cent = close_cent[i]
                curr_triangles = triangles[i]
                # Compute strategy's centrality scores for each node
                for node in G.nodes():
                    our_node_mapping[node] = alpha*curr_eig_cent[node]+beta*curr_close_cent[node]+(1-alpha-beta)*curr_triangles[node]
                seeds = nlargest(n, our_node_mapping.items(), key=operator.itemgetter(1))
                nodes = [j[0] for j in seeds]
                results = compare_all(adj_lists[i], nodes, strategies[i])
                for res in results:
                    score += res["Best"]
                    num_beat += int(res["Best"] > res["Test"])
            scores.append(score)
            num_beats.append(num_beat)
    max_beats = np.max(num_beats)
    print(max_beats)
    most_beats = [(scores[i], i) for i in range(len(num_beats)) if num_beats[i] >= max_beats]
    best = max(most_beats, key=lambda x: x[0])
    best_idx = best[1]
    return num_beats, scores, best_idx, coeffs[best_idx]

In [125]:
# team_names = ["LMNOP", "ModeloTime", "NetworkX", "_EAA", "_eek", "alley5", "pareto", \
#     "Charlemagne", "NoSleepAtAll", "_NKS", "_pe2", "LongTailedBeavers", "covid_23"]

# team_names = ["_EAA", "ModeloTime", "NoSleepAtAll", "alley5"]

team_names = ["_EAA", "NoSleepAtAll", "alley5"]

# Graph filenames
er_graph_filenames = ["RR.10.12", "RR.10.13", "RR.10.14"]
pa_graph_filenames = ["RR.10.23", "RR.10.24"]
ssbm_graph_filenames = ["RR.10.32", "RR.10.33", "RR.10.34"]
caltech_graph_filenames = ["RR.10.42", "RR.20.43", "RR.15.44"]
snap_graph_filenames = ["RR.10.52", "RR.10.53", "RR.10.54"]

# Strategies
strategies = []
for graph_filename in caltech_graph_filenames:
    er_strategies = []
    strategy_filenames = retrieve_strat_filenames(graph_filename, team_names)
    strats = get_strategies(strategy_filenames)
    for strat in strats:
        er_strategies.append(list(strat.values())[0][0][:10])
    strategies.append(er_strategies)
graphs, adj_lists = get_graphs(caltech_graph_filenames)
num_beats, scores, best_idx, best_coeffs = best_strategy(graphs, adj_lists, strategies)

KeyboardInterrupt: 

In [ ]:
print(num_beats)
print(scores)
print(best_idx)
print(best_coeffs)
print(scores[best_idx])
print(num_beats[best_idx])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 0, 1, 1, 1, 1, 1, 1, 1, 2, 0, 0, 1, 1, 1, 1, 1, 1, 2, 0, 0, 0, 1, 1, 1, 1, 1, 2, 0, 0, 0, 0, 1, 1, 1, 1, 2, 0, 0, 0, 0, 0, 1, 1, 1, 2, 0, 0, 0, 0, 0, 0, 1, 1, 2, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1072, 1072, 1072, 1072, 1072, 1072, 1072, 1072, 1072, 1080, 1072, 1072, 1072, 1072, 1072, 1072, 1072, 1072, 1301, 91, 1072, 1072, 1072, 1072, 1072, 1072, 1072, 1272, 91, 91, 1072, 1072, 1072, 1072, 1072, 1072, 1272, 91, 91, 91, 1072, 1072, 1072, 1072, 1072, 1517, 91, 91, 91, 91, 1072, 1072, 1072, 1072, 1517, 91, 91, 91, 91, 91, 1072, 1072, 1072, 1506, 91, 91, 91, 91, 91, 91, 1072, 1072, 1476, 91, 91, 91, 91, 91, 91, 91, 1072, 1346, 90, 91, 91, 91, 91, 91, 91, 91, 1346, 90, 90, 90, 91, 91, 91, 91, 91, 91]
45
(0.4444444444444444, 0.5555555555555556)
1517
2
